## Demo Scenario
We are going to be analyzing some rental data

In [25]:
from midas import Midas
m = Midas()

found name m


## Basic Profiling and Visualizations
When loading data into Midas, we show a basic profiler on the side.

In [26]:
contacts_df = m.read_table("/Users/yifanwu/Dev/midas/notebooks/rental/contacts.csv")

found name contacts_df
+ Addign df contacts_df


In [24]:
contacts_df.table._formats

{}

In [16]:
# [MIDAS] You selected the following projection on contacts_df at time 11/29/2019, 6:21:26 PM
contacts_df_id_host_anon = contacts_df.select(['id_host_anon'])

found name contacts_df_id_host_anon
+ Addign df contacts_df_id_host_anon


In [11]:
# [MIDAS] You selected the following projection on contacts_df at time 11/29/2019, 6:14:12 PM
contacts_df_id_host_anon = contacts_df.select(['id_host_anon'])

found name contacts_df_id_host_anon
+ Addign df contacts_df_id_host_anon


In [3]:
# [MIDAS] You selected the following projection on contacts_df at time 11/29/2019, 5:06:02 PM
contacts_df_id_guest_anon = contacts_df.select(['id_guest_anon'])

found name contacts_df_id_guest_anon
+ Addign df contacts_df_id_guest_anon


In [20]:
contacts_df_id_guest_anon._formats

NameError: name 'contacts_df_id_guest_anon' is not defined

In [19]:
m.dfs['contacts_df_id_host_anon'].df

id_host_anon
5426897d-960d-4013-9e38-606ae746793c
f30417c5-6df4-45ac-bfc2-6ad1cce398ab
13cbf50a-3272-45d4-9866-a06b6ea1b99a
01614601-d5a4-4776-ab9b-c10d3b865bf0
f2fed6f3-4c5c-453d-9e64-37c62b8bd06d
90334ef3-f489-45a0-89e0-d18d370e4a1c
10dc3fc4-694f-44b2-aaac-9dbace0ebabd
7e0696cb-6321-4d60-a8e9-793ee14724fd
1c990fca-5b09-4eca-bdcf-d9e923112f04
2bd81140-3d71-4fd9-add3-5998f9263c57
... (27877 rows omitted)

## Interactive Charts are Generated

In [ ]:
# now let's say that there is an interesting chart the user wants to look at
contact_channel_first_df = contacts_df.select(["contact_channel_first"])

In [ ]:
interactions_df = contacts_df.select(["m_interactions"])

### Oberservation
From the chart we can see that there must be some data scleaning issues.

## Access Selections Stream

In [ ]:
contact_channel_stream = contact_channel_first_df.get_stream()
contact_channel_stream

In [ ]:
contact_channel_stream.current

In [ ]:
contacts_df.project([contacts_df.ts_booking_at]).pandas_value

In [ ]:
room_type_df = listings_df.project([listings_df.room_type]).assign("room_type_df")

In [ ]:
# note the code below does not yet work!

In [ ]:
total_reviews_df = listings_df.project([listings_df.total_reviews]).assign("total_reviews_df")


## Reactive Cells
Cells on state that will change will be reactively called

In [ ]:
def transform(predicate):
    m.ui_comm.send_debug_msg(f"Transform {predicate}")
    new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
    new_mdf.project([new_mdf.m_guests]).assign("m_guests")

contact_channel_stream.add_callback(transform)

In [ ]:
new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
m_guests = new_mdf.project([new_mdf.m_guests]).assign("m_guests")
m_guests.pandas_value

In [ ]:
contact_channel_stream.ref_to_predicate_list

In [ ]:
m.event_loop.tick_funcs['contact_channel_first_df'] = []

In [ ]:
p = contact_channel_stream.current
m.event_loop.tick('contact_channel_first_df', p)

In [ ]:
predicate = contact_channel_stream.current
new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
m_guests = new_mdf.project([new_mdf.m_guests]).assign("m_guests") 


## Working with more than one table

In [ ]:
listings = m.read_table("/Users/yifanwu/Dev/midas/notebooks/rental/listings.csv")
users = m.read_table("/Users/yifanwu/Dev/midas/notebooks/rental/users.csv")